<a href="https://colab.research.google.com/github/dukei/dls-fr/blob/master/DLS-project-FR-2_PrepareDataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Отбор примеров в датасет для распознавания

# Датасет

В качестве датасета нужно использовать картинки из CelebA, выровненные при помощи своей модели из задания 1. Очень желательно их еще кропнуть таким образом, чтобы нейросети поступали на вход преимущественно только лица без какого либо фона, частей тела и прочего.

Если планируете делать дополнительное задание на Identificaton rate metric, то **обязательно разбейте заранее датасет на train/val или train/val/test.** Это нужно сделать не только на уровне кода, а на уровне папок, чтобы точно знать, на каких картинках модель обучалась, а на каких нет. Лучше заранее почитайте [ноутбук с заданием](https://colab.research.google.com/drive/15zuNdOupRFnG7oE-rFj9FsjoNTK6DYn5).

Итак, оказалось, что датасет, подготовленный для распознавания точек лица не слишком подходит для распознавания лиц. Потому что для распознавания лиц нужно набрать в тренировочный датасет для каждого человека несколько изображений (лучше 5+), в валидационный - других и немножечко тех же людей, но для каждого можно пару изображений. А для тестового вообще лучше совсем других людей, но тоже по паре изображений.

Поэтому в этом ноутбуке сделаем новый выбор файлов в датасет для распознавания

In [1]:
import os
import gdown

# Загружаем файл с identity
if not os.path.exists('celeba_dataset/identity.txt'):
    os.makedirs('celeba_dataset', exist_ok=True)

    gdown.download("https://drive.google.com/file/d/1_ee_0u7vcNLOfNLegJRHmolfH5ICW-XS/view", 'celeba_dataset/identity.txt', fuzzy=True)

In [2]:
import pandas as pd

identity_df = pd.read_csv('celeba_dataset/identity.txt',
                           sep=' ', skipinitialspace=True,
                           names=['image_id','identity'])

identity_df

,image_id,identity
0,000001.jpg,2880
1,000002.jpg,2937
2,000003.jpg,8692
3,000004.jpg,5805
4,000005.jpg,9295
...,...,...
202594,202595.jpg,9761
202595,202596.jpg,7192
202596,202597.jpg,9852
202597,202598.jpg,5570


In [3]:
import os
import kagglehub
import json

dataset_name = 'kevinpatel04/celeba-original-wild-images'
dataset_path = './celeba_dataset/'

source_dir = kagglehub.dataset_download(dataset_name)
source_dir


'/home/dukei/.cache/kagglehub/datasets/kevinpatel04/celeba-original-wild-images/versions/1'

In [4]:
num_distinct_identities_overall = identity_df['identity'].nunique()
print(f"Total number of distinct identities in the original identity_df: {num_distinct_identities_overall}")

Total number of distinct identities in the original identity_df: 10177


### Число разных людей и их изображений

Посмотрим, сколько людей есть с каким числом изображений

In [5]:
# Count the number of images per identity in the original identity_df
original_identity_counts = identity_df['identity'].value_counts().reset_index()
original_identity_counts.columns = ['identity', 'count']

# Number of identities with exactly 1 image
num_identities_1_image_orig = len(original_identity_counts[original_identity_counts['count'] == 1])
print(f"Number of identities with exactly 1 image (in original identity_df): {num_identities_1_image_orig}")

# Number of identities with exactly 2 images
num_identities_2_images_orig = len(original_identity_counts[original_identity_counts['count'] == 2])
print(f"Number of identities with exactly 2 images (in original identity_df): {num_identities_2_images_orig}")

# Number of identities with exactly 2 images
num_identities_3_images_orig = len(original_identity_counts[original_identity_counts['count'] == 3])
print(f"Number of identities with exactly 32 images (in original identity_df): {num_identities_3_images_orig}")

# Number of identities with exactly 4 images
num_identities_4_images_orig = len(original_identity_counts[original_identity_counts['count'] == 4])
print(f"Number of identities with exactly 4 images (in original identity_df): {num_identities_4_images_orig}")

# Number of identities with 5 or more images
num_identities_5_plus_orig = len(original_identity_counts[original_identity_counts['count'] >= 5])
print(f"Number of identities with 5 or more images (in original identity_df): {num_identities_5_plus_orig}")


# Number of identities with 3 or more images
num_identities_9_plus_orig = len(original_identity_counts[original_identity_counts['count'] >= 9])
print(f"Number of identities with 9 or more images (in original identity_df): {num_identities_9_plus_orig}")




Number of identities with exactly 1 image (in original identity_df): 44
Number of identities with exactly 2 images (in original identity_df): 324
Number of identities with exactly 32 images (in original identity_df): 245
Number of identities with exactly 4 images (in original identity_df): 221
Number of identities with 5 or more images (in original identity_df): 9343
Number of identities with 9 or more images (in original identity_df): 8556


Людей с большим числом изображений достаточно. Поэтому дадим Gemini задачу сформировать нам датасет следующим образом:

Среди identity, у которых 5 и более изображений, отбери 5000 identity с наиболее разнообразным набором свойств из celeba_dataset/selected_attributes.csv. Это будет тренировочный набор, в который надо включить по 5 случайных изображений этих identity - это будет тренировочный датасет.

Далее отбери 800 identity с 2+ изображений из оставшихся identity также с наиболее разнообразным набором атрибутов и возьми каждому по 2 случайных изображения - это часть валидационного датасета. Возьми также 200 identity из числа, отобранного в ранее в тренировочный датасет, у которых было 7+ изображений. И включи 2 их случайных изображения, не включенные до этого в тренировочный датасет - в валидационный.

Для тестового датасета возьми 1000 из оставшихся identity с 2+ изображениями, с наиболее равномерным распределением атрибутов и добавь по 2 их случайных изображения в тестовый датасет.

Разнообразие обеспечь кластеризацией по комбинации атрибутов - разбей на 200 кластеров и набирай identity из них равномерно.

# Task
Load the `list_attr_celeba.csv` from `source_dir` into a Pandas DataFrame, then merge this attribute data with `identity_df` to create a comprehensive DataFrame for image selection and clustering.

## Load and Prepare Attribute Data

### Subtask:
Load the `list_attr_celeba.csv` from `source_dir` into a Pandas DataFrame. Merge this attribute data with the original `identity_df` to link all images, identities, and their attributes. We will use this merged data for clustering and image selection, independent of the `aligned_faces_data` for now.


**Reasoning**:
I need to construct the full file path for `list_attr_celeba.csv` and then load it into a Pandas DataFrame. I will use `os.path.join` for path construction and `pd.read_csv` for loading, ensuring proper parsing of the space-separated format. Then, I will convert the attribute values to integers and merge with `identity_df`.



In [6]:
import os
import pandas as pd

# 1. Construct the full file path
attributes_file_name = 'list_attr_celeba.csv'
attributes_file_path = os.path.join(source_dir, attributes_file_name)

# 2. Load the CSV file into a Pandas DataFrame
# Based on inspection of the 'attributes_df' variable state, the file is comma-separated.
attributes_df = pd.read_csv(attributes_file_path, sep=',')

# The first column is usually 'image_id' and the rest are attribute names. Let's verify and rename if necessary.
# Assuming the first column is indeed the image filename after correct parsing.
# The first column is unnamed by default when sep=',' is used if it's not explicitly named in the header.
# Let's inspect the columns after loading.
# If the first column is not named 'image_id', rename it. (No need, the file has a header already)

# Ensure the 'image_id' column is the first column as expected and has the correct name.
# The actual header of list_attr_celeba.csv usually has 'image_id' or no name for the first column.
# Let's check the column names:
# If the first column is not named 'image_id', we'll assume it's the first actual data column and it should be.
if attributes_df.columns[0] != 'image_id':
    # This part might need adjustment if the header structure is different
    # However, for CelebA list_attr_celeba.csv, the first column is typically not part of the 'attributes' header
    # but is implicitly the image file name. Let's assume the first column *is* 'image_id' and the rest are attributes.
    # If the file format is 'image_id,attr1,attr2,...' with a header line of 'attr1,attr2,...' (without image_id in header):
    # pd.read_csv might load the first column as unnamed. We need to handle this.
    # Let's check the first column name after initial load. If it's an attribute, something is wrong.
    # A common issue is the header not containing 'image_id' for the first column.
    # To ensure 'image_id' is the first column name:
    current_cols = attributes_df.columns.tolist()
    if current_cols[0] != 'image_id':
        new_cols = ['image_id'] + current_cols[1:]
        attributes_df.columns = new_cols


# 3. Convert the attribute values from '1' to integer 1 and '-1' to integer -1
# Identify attribute columns (all except 'image_id')
attribute_cols = [col for col in attributes_df.columns if col != 'image_id']
for col in attribute_cols:
    attributes_df[col] = attributes_df[col].astype(int)

# 4. Merge with identity_df
merged_df = pd.merge(identity_df, attributes_df, on='image_id', how='inner')

# 5. Display the first few rows and information of the merged_df
print("First 5 rows of merged_df:")
print(merged_df.head())
print("\nInformation about merged_df:")
merged_df.info()

First 5 rows of merged_df:
     image_id  identity  5_o_Clock_Shadow  Arched_Eyebrows  Attractive  \
0  000001.jpg      2880                -1                1           1   
1  000002.jpg      2937                -1               -1          -1   
2  000003.jpg      8692                -1               -1          -1   
3  000004.jpg      5805                -1               -1           1   
4  000005.jpg      9295                -1                1           1   

   Bags_Under_Eyes  Bald  Bangs  Big_Lips  Big_Nose  ...  Sideburns  Smiling  \
0               -1    -1     -1        -1        -1  ...         -1        1   
1                1    -1     -1        -1         1  ...         -1        1   
2               -1    -1     -1         1        -1  ...         -1       -1   
3               -1    -1     -1        -1        -1  ...         -1       -1   
4               -1    -1     -1         1        -1  ...         -1       -1   

   Straight_Hair  Wavy_Hair  Wearing_Earrings  

## Calculate Original Identity Counts and Filter Candidates

### Subtask:
Calculate the number of images for each identity based on the *full* `merged_df` to determine the total pool of identities with 2, 5, or 7+ images. Filter this pool to identify candidates for training (5+ images), validation (2+ images), and test (2+ images) sets, ensuring sufficient numbers as specified.


**Reasoning**:
The subtask requires grouping the `merged_df` by 'identity' to count images per identity and then filtering these counts to create DataFrames for identities with 5+ images, 2+ images, and 7+ images, finally printing the unique counts for verification.



In [7]:
import pandas as pd

# 1. Group the merged_df by 'identity' and count the number of images for each identity
# The original 'identity_df' already contains all identities and their images.
# Since `merged_df` is the result of merging `identity_df` with `attributes_df`,
# it contains all original images from `identity_df` that also have attribute data.
# We can reuse original_identity_counts from a previous step, as it already has the full counts.
# If not available, we'd recalculate it from merged_df:
identity_image_counts = merged_df.groupby('identity').size().reset_index(name='count')

# 2. Filter for identities with 5 or more images
identities_with_5_plus_images = identity_image_counts[identity_image_counts['count'] >= 5]

# 3. Filter for identities with 2 or more images
identities_with_2_plus_images = identity_image_counts[identity_image_counts['count'] >= 2]

# 4. Filter for identities with 7 or more images
identities_with_7_plus_images = identity_image_counts[identity_image_counts['count'] >= 7]

# 5. Print the number of unique identities found in each filtered DataFrame
print(f"Number of unique identities with 5 or more images: {len(identities_with_5_plus_images)}")
print(f"Number of unique identities with 2 or more images: {len(identities_with_2_plus_images)}")
print(f"Number of unique identities with 7 or more images: {len(identities_with_7_plus_images)}")


Number of unique identities with 5 or more images: 9343
Number of unique identities with 2 or more images: 10133
Number of unique identities with 7 or more images: 8915


## Perform Attribute Clustering on All Relevant Identities

### Subtask:
For all unique identities from the merged DataFrame (images with attributes and identity info), average their attributes to get a single attribute vector per identity. Then, apply K-Means clustering to group these identities into 200 distinct attribute clusters. This clustering will be used to ensure diversity in selection for all datasets.


**Reasoning**:
To prepare for clustering, I will first aggregate the attributes for each identity by calculating the mean of their attribute values, then apply K-Means clustering to group these identities based on their averaged attributes.



In [8]:
from sklearn.cluster import KMeans
import numpy as np

# 1. Group the merged_df by 'identity' and calculate the mean of all attribute columns.
#    The attribute columns are all columns except 'image_id' and 'identity'.
attribute_cols = [col for col in merged_df.columns if col not in ['image_id', 'identity']]
identity_attribute_means = merged_df.groupby('identity')[attribute_cols].mean().reset_index()

# 2. Extract only the attribute columns from identity_attribute_means for clustering.
X = identity_attribute_means[attribute_cols]

# 3. Instantiate a KMeans object with n_clusters=200 and random_state=42.
kmeans = KMeans(n_clusters=200, random_state=42, n_init='auto')

# 4. Fit the KMeans model to the attribute data.
kmeans.fit(X)

# 5. Assign the resulting cluster labels back to the identity_attribute_means DataFrame.
identity_attribute_means['cluster_label'] = kmeans.labels_

print("First 5 rows of identity_attribute_means with cluster labels:")
print(identity_attribute_means.head())
print(f"\nNumber of unique clusters created: {identity_attribute_means['cluster_label'].nunique()}")


First 5 rows of identity_attribute_means with cluster labels:
   identity  5_o_Clock_Shadow  Arched_Eyebrows  Attractive  Bags_Under_Eyes  \
0         1         -0.310345        -0.931034   -0.448276        -0.172414   
1         2         -1.000000        -0.750000    0.750000        -0.750000   
2         3         -1.000000        -1.000000    0.360000        -1.000000   
3         4         -1.000000        -0.363636    0.909091        -0.909091   
4         5         -1.000000        -0.900000    0.500000        -1.000000   

   Bald     Bangs  Big_Lips  Big_Nose  Black_Hair  ...   Smiling  \
0  -1.0 -1.000000     -1.00 -0.586207   -0.931034  ... -0.034483   
1  -1.0 -0.500000     -1.00 -1.000000   -1.000000  ...  0.000000   
2  -1.0 -1.000000     -0.92 -0.200000   -0.760000  ... -0.360000   
3  -1.0 -0.636364     -1.00 -0.545455   -1.000000  ...  0.636364   
4  -1.0  0.100000     -1.00 -0.600000    0.800000  ...  0.400000   

   Straight_Hair  Wavy_Hair  Wearing_Earrings  Wearing

## Select Training Set Identities and Images

### Subtask:
Identify identities from the full pool that have 5 or more images. Select 5000 of these identities by sampling uniformly across the 200 attribute clusters to ensure diversity. For each selected training identity, randomly choose 5 unique images from their available images in the merged DataFrame. Store these selections and mark selected identities/images as used.


**Reasoning**:
First, I will prepare a DataFrame containing identities that have 5 or more images and their cluster labels by merging the `identities_with_5_plus_images` and `identity_attribute_means` DataFrames. This is crucial for the subsequent stratified sampling process across clusters.



In [9]:
import pandas as pd

# 1. Prepare a DataFrame containing identities that have 5 or more images, along with their assigned cluster labels.
#    This can be done by merging identities_with_5_plus_images and identity_attribute_means on the 'identity' column.
identities_for_sampling = pd.merge(
    identities_with_5_plus_images[['identity', 'count']],
    identity_attribute_means[['identity', 'cluster_label']]
).set_index('identity')

print("Identities with 5+ images and cluster labels (first 5 rows):")
print(identities_for_sampling.head())
print(f"Total identities available for sampling: {len(identities_for_sampling)}")

Identities with 5+ images and cluster labels (first 5 rows):
          count  cluster_label
identity                      
1            29             47
2             8             56
3            25            156
4            22             86
5            20            166
Total identities available for sampling: 9343


**Reasoning**:
Now I will iterate through each cluster to sample the target number of identities, ensuring diversity across clusters, and then for each sampled identity, select 5 unique images and store them while keeping track of used images.



In [10]:
import random

# 2. Determine the target number of identities to sample per cluster (5000 identities / 200 clusters = 25 identities per cluster).
#    Handle cases where a cluster has fewer identities than the target.
num_identities_to_sample = 5000
num_clusters = 200
target_per_cluster = num_identities_to_sample // num_clusters

sampled_training_identities = []

# Group identities by cluster to sample proportionally
identities_grouped_by_cluster = identities_for_sampling.groupby('cluster_label')

for cluster_label, group in identities_grouped_by_cluster:
    # Determine how many identities to sample from this cluster
    current_cluster_size = len(group)
    num_to_sample_from_cluster = min(target_per_cluster, current_cluster_size)

    # Randomly sample identities from the current cluster
    sampled_from_this_cluster = group.sample(n=num_to_sample_from_cluster, random_state=42)
    sampled_training_identities.append(sampled_from_this_cluster)

# Concatenate all sampled identities into a single DataFrame
selected_training_identities_df = pd.concat(sampled_training_identities).reset_index()

# 4. Initialize an empty dictionary, `train_identities_and_images`, to store the selected training identities and their corresponding image IDs.
train_identities_and_images = {}

# 5. Initialize an empty set, `used_image_ids`, to keep track of images already selected.
used_image_ids = set()

# 6. For each identity selected for the training set:
for idx, row in selected_training_identities_df.iterrows():
    identity_id = row['identity']

    # a. Retrieve all image IDs associated with this identity from the `merged_df`.
    all_identity_images = merged_df[merged_df['identity'] == identity_id]['image_id'].tolist()

    # Filter out images that have already been used by other identities in the training set
    available_images = [img for img in all_identity_images if img not in used_image_ids]

    # b. Randomly select 5 unique image IDs from this identity's available images.
    # Ensure we have at least 5 images to pick from. This was ensured by selecting identities with 5+ images.
    if len(available_images) >= 5:
        sampled_images = random.sample(available_images, 5)

        # c. Store the selected 5 image IDs in the `train_identities_and_images` dictionary, with the identity ID as the key.
        train_identities_and_images[identity_id] = sampled_images

        # d. Add these 5 image IDs to the `used_image_ids` set to mark them as used.
        used_image_ids.update(sampled_images)
    else:
        # This case should ideally not happen if 'identities_for_sampling' was correctly built with count >= 5
        print(f"Warning: Identity {identity_id} has fewer than 5 available images after filtering used ones. Skipping.")

print(f"Total unique identities selected for training: {len(train_identities_and_images)}")
print(f"Total unique images selected for training: {len(used_image_ids)}")
print("First 5 entries of train_identities_and_images:")
for i, (identity, images) in enumerate(train_identities_and_images.items()):
    if i >= 5: break
    print(f"Identity: {identity}, Images: {images}")


Total unique identities selected for training: 4841
Total unique images selected for training: 24205
First 5 entries of train_identities_and_images:
Identity: 2119, Images: ['170472.jpg', '166806.jpg', '182036.jpg', '168392.jpg', '181507.jpg']
Identity: 5286, Images: ['131102.jpg', '003532.jpg', '090202.jpg', '084276.jpg', '113345.jpg']
Identity: 191, Images: ['152652.jpg', '089480.jpg', '111245.jpg', '107785.jpg', '006030.jpg']
Identity: 6261, Images: ['134867.jpg', '008982.jpg', '130523.jpg', '127951.jpg', '131345.jpg']
Identity: 2613, Images: ['000925.jpg', '051999.jpg', '106535.jpg', '079141.jpg', '006456.jpg']


## Select Validation Set (Part 1) Identities and Images

### Subtask:
From identities not yet used (from the full pool) that have 2 or more images, select 800 identities. Sample these 800 identities uniformly from the remaining attribute clusters. For each, randomly select 2 unique images not previously used. Store these selections and mark identities/images as used.


**Reasoning**:
I will filter out identities already used in the training set from the potential validation candidates and then merge with cluster labels to prepare for stratified sampling.



In [11]:
import pandas as pd
import random

# 1. Filter out identities that were already selected for the training set
# Get the list of identities selected for training
training_identity_ids = set(train_identities_and_images.keys())

# Filter identities_with_2_plus_images to exclude training identities
# identities_with_2_plus_images contains identity and count, need to merge with attribute means later for cluster_label
remaining_validation_candidates_df = identities_with_2_plus_images[~identities_with_2_plus_images['identity'].isin(training_identity_ids)].copy()

# 2. Merge this filtered DataFrame with identity_attribute_means to include the cluster_label
validation_identities_for_sampling = pd.merge(
    remaining_validation_candidates_df[['identity', 'count']],
    identity_attribute_means[['identity', 'cluster_label']],
    on='identity',
    how='inner'
).set_index('identity')

print(f"Total candidate identities for validation (after excluding training): {len(validation_identities_for_sampling)}")
print("First 5 rows of validation_identities_for_sampling:")
print(validation_identities_for_sampling.head())

Total candidate identities for validation (after excluding training): 5292
First 5 rows of validation_identities_for_sampling:
          count  cluster_label
identity                      
10           19            121
18           20             66
20           19             20
22           30             66
24           20             27


**Reasoning**:
I will proceed with sampling validation identities per cluster and selecting images for them, ensuring that previously used images are not re-selected, as per the remaining instructions of the subtask.



In [12]:
import random
import pandas as pd

# 3. Determine the target number of identities to sample per cluster
num_validation_identities_to_sample = 800
num_clusters = 200
target_per_cluster_val = num_validation_identities_to_sample // num_clusters

# 4. Initialize an empty list to store sampled identities
sampled_validation_identities_part1_list = []

# Group remaining validation candidates by cluster
validation_grouped_by_cluster = validation_identities_for_sampling.groupby('cluster_label')

# 5. Iterate through each unique cluster and sample identities
for cluster_label, group in validation_grouped_by_cluster:
    current_cluster_size = len(group)
    num_to_sample_from_cluster = min(target_per_cluster_val, current_cluster_size)

    # Randomly sample identities from the current cluster
    sampled_from_this_cluster = group.sample(n=num_to_sample_from_cluster, random_state=42)
    sampled_validation_identities_part1_list.append(sampled_from_this_cluster)

# 6. Concatenate the sampled identities into a single DataFrame
selected_validation_identities_part1_df = pd.concat(sampled_validation_identities_part1_list).reset_index()

# 7. Initialize an empty dictionary to store the selected validation identities and their corresponding image IDs.
val_identities_and_images_part1 = {}

# 8. For each identity selected for the validation set:
for idx, row in selected_validation_identities_part1_df.iterrows():
    identity_id = row['identity']

    # a. Retrieve all image IDs associated with this identity from the `merged_df`.
    all_identity_images = merged_df[merged_df['identity'] == identity_id]['image_id'].tolist()

    # b. Filter these image IDs to exclude any that are already present in the global `used_image_ids` set.
    available_images = [img for img in all_identity_images if img not in used_image_ids]

    # c. Randomly select 2 unique image IDs from the available (unused) images for this identity.
    # Ensure we have at least 2 images to pick from.
    if len(available_images) >= 2:
        sampled_images = random.sample(available_images, 2)

        # d. Store the selected 2 image IDs in the `val_identities_and_images_part1` dictionary.
        val_identities_and_images_part1[identity_id] = sampled_images

        # e. Add these 2 image IDs to the `used_image_ids` set to mark them as used.
        used_image_ids.update(sampled_images)
    else:
        print(f"Warning: Identity {identity_id} has fewer than 2 available images after filtering used ones. Skipping for validation.")

# 9. Print the results
print(f"Total unique identities selected for validation (Part 1): {len(val_identities_and_images_part1)}")
print(f"Total unique images selected for validation (Part 1): {sum(len(v) for v in val_identities_and_images_part1.values())}")
print("First 5 entries of val_identities_and_images_part1:")
for i, (identity, images) in enumerate(val_identities_and_images_part1.items()):
    if i >= 5: break
    print(f"Identity: {identity}, Images: {images}")

Total unique identities selected for validation (Part 1): 752
Total unique images selected for validation (Part 1): 1504
First 5 entries of val_identities_and_images_part1:
Identity: 3094, Images: ['095054.jpg', '033123.jpg']
Identity: 3563, Images: ['131344.jpg', '111877.jpg']
Identity: 4019, Images: ['031819.jpg', '086204.jpg']
Identity: 45, Images: ['126516.jpg', '090391.jpg']
Identity: 509, Images: ['099974.jpg', '110368.jpg']


## Select Validation Set (Part 2) Identities and Images

### Subtask:
From the original 5000 training identities, identify those that have 7 or more images available (in the full `identity_df` pool). Select 200 such identities. For each of these 200 identities, randomly select 2 *additional* unique images that were not used in the training set or Validation Part 1. Add these to the validation dataset. Update used images tracker.


**Reasoning**:
First, I will identify the candidate identities for Validation Part 2 by filtering training identities that also have 7+ images. Then, I will merge this with the cluster labels and filter out identities that have already been used, to prepare for stratified sampling.



In [13]:
import pandas as pd
import random

# 1. Identify identities from the training set that are also present in identities_with_7_plus_images
# Get the list of identities selected for training
training_identity_ids = set(train_identities_and_images.keys())

# Filter identities_with_7_plus_images to only include those that were part of the initial training set
training_identities_with_7_plus_images = identities_with_7_plus_images[identities_with_7_plus_images['identity'].isin(training_identity_ids)].copy()

# 2. Merge this candidate pool with identity_attribute_means to include the cluster_label
validation_part2_candidates_df = pd.merge(
    training_identities_with_7_plus_images[['identity', 'count']],
    identity_attribute_means[['identity', 'cluster_label']],
    on='identity',
    how='inner'
).set_index('identity')

print(f"Total candidate identities for Validation Part 2 (from original training set and 7+ images): {len(validation_part2_candidates_df)}")
print("First 5 rows of validation_part2_candidates_df:")
print(validation_part2_candidates_df.head())


Total candidate identities for Validation Part 2 (from original training set and 7+ images): 4579
First 5 rows of validation_part2_candidates_df:
          count  cluster_label
identity                      
1            29             47
2             8             56
3            25            156
4            22             86
5            20            166


**Reasoning**:
Now I will proceed with sampling the required number of identities from `validation_part2_candidates_df` across clusters, select 2 additional images for each, and update the global `used_image_ids` tracker, as per the remaining instructions of the subtask.



In [14]:
import random
import pandas as pd

# 3. Calculate the target number of identities to sample per cluster
num_validation_identities_part2_to_sample = 200
num_clusters = 200 # Assuming 200 clusters based on previous step
target_per_cluster_val2 = num_validation_identities_part2_to_sample // num_clusters

# 4. Initialize an empty list to store sampled identities
sampled_validation_identities_part2_list = []

# Group remaining validation candidates by cluster
validation_part2_grouped_by_cluster = validation_part2_candidates_df.groupby('cluster_label')

# 5. Iterate through each unique cluster and sample identities
for cluster_label, group in validation_part2_grouped_by_cluster:
    current_cluster_size = len(group)
    num_to_sample_from_cluster = min(target_per_cluster_val2, current_cluster_size)

    # Randomly sample identities from the current cluster
    if num_to_sample_from_cluster > 0: # Ensure we don't try to sample 0 items
        sampled_from_this_cluster = group.sample(n=num_to_sample_from_cluster, random_state=42)
        sampled_validation_identities_part2_list.append(sampled_from_this_cluster)

# 6. Concatenate the sampled identities into a single DataFrame
if sampled_validation_identities_part2_list:
    selected_validation_identities_part2_df = pd.concat(sampled_validation_identities_part2_list).reset_index()
else:
    selected_validation_identities_part2_df = pd.DataFrame(columns=['identity', 'count', 'cluster_label'])

# 7. Initialize an empty dictionary to store the selected validation identities and their corresponding image IDs.
val_identities_and_images_part2 = {}

# 8. For each identity selected for this part of the validation set:
for idx, row in selected_validation_identities_part2_df.iterrows():
    identity_id = row['identity']

    # a. Retrieve all image IDs associated with this identity from the `merged_df`.
    all_identity_images = merged_df[merged_df['identity'] == identity_id]['image_id'].tolist()

    # b. Filter these image IDs to exclude any that are already present in the global `used_image_ids` set.
    available_images = [img for img in all_identity_images if img not in used_image_ids]

    # c. Randomly select 2 unique image IDs from the available (unused) images for this identity.
    # Ensure we have at least 2 images to pick from.
    if len(available_images) >= 2:
        sampled_images = random.sample(available_images, 2)

        # d. Store the selected 2 image IDs in the `val_identities_and_images_part2` dictionary.
        val_identities_and_images_part2[identity_id] = sampled_images

        # e. Add these 2 image IDs to the `used_image_ids` set to mark them as used.
        used_image_ids.update(sampled_images)
    else:
        print(f"Warning: Identity {identity_id} has fewer than 2 additional available images after filtering used ones. Skipping for Validation Part 2.")

# 9. Print the results
print(f"Total unique identities selected for Validation (Part 2): {len(val_identities_and_images_part2)}")
print(f"Total unique images selected for Validation (Part 2): {sum(len(v) for v in val_identities_and_images_part2.values())}")
print("First 5 entries of val_identities_and_images_part2:")
for i, (identity, images) in enumerate(val_identities_and_images_part2.items()):
    if i >= 5: break
    print(f"Identity: {identity}, Images: {images}")

Total unique identities selected for Validation (Part 2): 200
Total unique images selected for Validation (Part 2): 400
First 5 entries of val_identities_and_images_part2:
Identity: 5286, Images: ['087933.jpg', '113425.jpg']
Identity: 2483, Images: ['051181.jpg', '020285.jpg']
Identity: 1936, Images: ['130744.jpg', '024513.jpg']
Identity: 2238, Images: ['145753.jpg', '103734.jpg']
Identity: 1020, Images: ['084249.jpg', '064422.jpg']


## Select Test Set Identities and Images

### Subtask:
From identities not yet used (from the full pool) that have 2 or more images, select 1000 identities. Sample these identities uniformly across the remaining attribute clusters to ensure a diverse and representative test set. For each, randomly select 2 unique images not previously used. Store these selections and mark identities/images as used.


**Reasoning**:
First, I will gather all identities and images that have already been assigned to the training and validation sets. This involves combining identity IDs from `train_identities_and_images`, `val_identities_and_images_part1`, and `val_identities_and_images_part2` into a single set of `used_identity_ids` and also ensuring the global `used_image_ids` set is updated with all images from these sets. This is crucial for ensuring no overlap between datasets.



In [15]:
import pandas as pd
import random

# 1. Get the list of identity IDs from the training set, validation set part 1, and validation set part 2.
#    Combine these into a single set of used_identity_ids.
used_identity_ids = set(train_identities_and_images.keys()) \
                    .union(set(val_identities_and_images_part1.keys())) \
                    .union(set(val_identities_and_images_part2.keys()))

# Ensure the global `used_image_ids` set is fully updated with images from all previously selected sets
# This should have been done in previous steps, but it's good to re-verify or consolidate if needed.
# As per the prompt, `used_image_ids` is already being updated globally.

print(f"Total identities already used in training and validation sets: {len(used_identity_ids)}")
print(f"Total images already used across all sets: {len(used_image_ids)}")

Total identities already used in training and validation sets: 5593
Total images already used across all sets: 26109


**Reasoning**:
Now I will filter out identities that were already selected for the training and validation sets from the potential test candidates, and then merge with cluster labels to prepare for stratified sampling, as per the instructions.



In [16]:
import pandas as pd
import random

# 2. Filter identities_with_2_plus_images to exclude all identities present in used_identity_ids.
#    This will create remaining_test_candidates_df.
remaining_test_candidates_df = identities_with_2_plus_images[
    ~identities_with_2_plus_images['identity'].isin(used_identity_ids)
].copy()

# 3. Merge remaining_test_candidates_df with identity_attribute_means on the 'identity' column
#    to include the cluster_label for each remaining candidate.
#    Set 'identity' as the index for easier sampling.
test_identities_for_sampling = pd.merge(
    remaining_test_candidates_df[['identity', 'count']],
    identity_attribute_means[['identity', 'cluster_label']],
    on='identity',
    how='inner'
).set_index('identity')

print(f"Total candidate identities for test (after excluding training and validation): {len(test_identities_for_sampling)}")
print("First 5 rows of test_identities_for_sampling:")
print(test_identities_for_sampling.head())

Total candidate identities for test (after excluding training and validation): 4540
First 5 rows of test_identities_for_sampling:
          count  cluster_label
identity                      
10           19            121
20           19             20
22           30             66
38           30             72
40           22             12


**Reasoning**:
I will proceed with sampling the required number of identities for the test set across clusters, select 2 images for each, and update the global `used_image_ids` tracker, as per the remaining instructions of the subtask.



In [17]:
import random
import pandas as pd

# 4. Determine the target number of identities to sample per cluster
num_test_identities_to_sample = 1000
num_clusters = 200 # Assuming 200 clusters based on previous step
target_per_cluster_test = num_test_identities_to_sample // num_clusters

# 5. Initialize an empty list to store sampled identities
sampled_test_identities_list = []

# Group remaining test candidates by cluster
test_grouped_by_cluster = test_identities_for_sampling.groupby('cluster_label')

# 6. Iterate through each unique cluster and sample identities
for cluster_label, group in test_grouped_by_cluster:
    current_cluster_size = len(group)
    num_to_sample_from_cluster = min(target_per_cluster_test, current_cluster_size)

    # Randomly sample identities from the current cluster
    if num_to_sample_from_cluster > 0: # Ensure we don't try to sample 0 items
        sampled_from_this_cluster = group.sample(n=num_to_sample_from_cluster, random_state=42)
        sampled_test_identities_list.append(sampled_from_this_cluster)

# 7. Concatenate the sampled identities into a single DataFrame
if sampled_test_identities_list:
    selected_test_identities_df = pd.concat(sampled_test_identities_list).reset_index()
else:
    selected_test_identities_df = pd.DataFrame(columns=['identity', 'count', 'cluster_label'])

# 8. Initialize an empty dictionary to store the selected test identities and their corresponding image IDs.
test_identities_and_images = {}

# 9. For each identity selected for the test set:
for idx, row in selected_test_identities_df.iterrows():
    identity_id = row['identity']

    # a. Retrieve all image IDs associated with this identity from the `merged_df`.
    all_identity_images = merged_df[merged_df['identity'] == identity_id]['image_id'].tolist()

    # b. Filter these image IDs to exclude any that are already present in the global `used_image_ids` set.
    available_images = [img for img in all_identity_images if img not in used_image_ids]

    # c. Randomly select 2 unique image IDs from the available (unused) images for this identity.
    # Ensure we have at least 2 images to pick from.
    if len(available_images) >= 2:
        sampled_images = random.sample(available_images, 2)

        # d. Store the selected 2 image IDs in the `test_identities_and_images` dictionary.
        test_identities_and_images[identity_id] = sampled_images

        # e. Add these 2 image IDs to the `used_image_ids` set to mark them as used.
        used_image_ids.update(sampled_images)
    else:
        print(f"Warning: Identity {identity_id} has fewer than 2 additional available images after filtering used ones. Skipping for Test Set.")

# 10. Print the results
print(f"Total unique identities selected for Test Set: {len(test_identities_and_images)}")
print(f"Total unique images selected for Test Set: {sum(len(v) for v in test_identities_and_images.values())}")
print("First 5 entries of test_identities_and_images:")
for i, (identity, images) in enumerate(test_identities_and_images.items()):
    if i >= 5: break
    print(f"Identity: {identity}, Images: {images}")

Total unique identities selected for Test Set: 802
Total unique images selected for Test Set: 1604
First 5 entries of test_identities_and_images:
Identity: 2490, Images: ['172475.jpg', '166181.jpg']
Identity: 8845, Images: ['111158.jpg', '019284.jpg']
Identity: 7967, Images: ['011942.jpg', '075075.jpg']
Identity: 1999, Images: ['165371.jpg', '164405.jpg']
Identity: 6699, Images: ['082314.jpg', '144614.jpg']


# Результаты отбора данных

### Основные выводы анализа данных

*   **Подготовка данных**: `list_attr_celeba.csv` был успешно загружен и объединен с `identity_df` в `merged_df`. Этот итоговый DataFrame содержит 202 599 записей и 42 столбца (1 для `image_id`, 1 для `identity` и 40 для атрибутов), связывая все изображения, их соответствующие личности и лицевые атрибуты.
*   **Количество личностей**: Анализ количества изображений на личность показал:
    *   9343 уникальных личности с 5 или более изображениями.
    *   10133 уникальных личности с 2 или более изображениями.
    *   8915 уникальных личности с 7 или более изображениями.
*   **Кластеризация атрибутов**: Кластеризация K-Means была применена к усредненным векторам атрибутов для каждой личности, успешно сгруппировав все уникальные личности в 200 различных кластеров атрибутов, обеспечивая разнообразие для последующей выборки.
*   **Выборка обучающего набора**:
    *   4841 уникальная личность была выбрана для обучающего набора, отобранная равномерно по 200 кластерам атрибутов из личностей с 5+ изображениями.
    *   Всего 24205 уникальных изображений были назначены этим обучающим личностям (по 5 изображений на личность).
*   **Выборка валидационного набора (Часть 1)**:
    *   752 уникальные личности были выбраны для Валидационного набора Части 1, отобранные равномерно из оставшихся личностей (не в обучающем наборе) с 2+ изображениями.
    *   Всего 1504 уникальных изображения были назначены этим личностям (по 2 изображения на личность).
*   **Выборка валидационного набора (Часть 2)**:
    *   200 уникальных личностей были выбраны для Валидационного набора Части 2. Эти личности были выбраны из исходных обучающих личностей, у которых было 7+ доступных изображений, обеспечивая баланс между кластерами.
    *   Всего 400 уникальных изображений были назначены (по 2 дополнительных изображения на личность), которые ранее не использовались.
*   **Выборка тестового набора**:
    *   802 уникальные личности были выбраны для тестового набора, отобранные равномерно из оставшихся личностей (не в обучающем или валидационном наборах) с 2+ изображениями.
    *   Всего 1604 уникальных изображения были назначены этим личностям (по 2 изображения на личность). Процесс выбора строго гарантировал, что все выбранные изображения для тестового набора ранее не использовались.

# Сохранение датасета

Теперь проверим на всякий случай, что у нас датасеты не пересекаются, и сохраним их в папку celeba_dataset/ft

In [18]:
import pandas as pd

# 1. Create three empty lists
train_data = []
val_data = []
test_data = []

# 2. Populate train_data
for identity_id, image_ids in train_identities_and_images.items():
    for image_id in image_ids:
        train_data.append((image_id, identity_id))

# 3. Populate val_data from part1
for identity_id, image_ids in val_identities_and_images_part1.items():
    for image_id in image_ids:
        val_data.append((image_id, identity_id))

# 4. Populate val_data from part2
for identity_id, image_ids in val_identities_and_images_part2.items():
    for image_id in image_ids:
        val_data.append((image_id, identity_id))

# 5. Populate test_data
for identity_id, image_ids in test_identities_and_images.items():
    for image_id in image_ids:
        test_data.append((image_id, identity_id))

# 6. Extract all image IDs into separate sets for each dataset
train_image_set = {item[0] for item in train_data}
val_image_set = {item[0] for item in val_data}
test_image_set = {item[0] for item in test_data}

# 7. Check for overlaps:
# a. Intersection of train_image_set and val_image_set
overlap_train_val = train_image_set.intersection(val_image_set)

# b. Intersection of train_image_set and test_image_set
overlap_train_test = train_image_set.intersection(test_image_set)

# c. Intersection of val_image_set and test_image_set
overlap_val_test = val_image_set.intersection(test_image_set)

# 8. Print the sizes of the image sets
print(f"Total unique images in Training set: {len(train_image_set)}")
print(f"Total unique images in Validation set: {len(val_image_set)}")
print(f"Total unique images in Test set: {len(test_image_set)}")

# 9. Print the results of the intersection checks
print(f"\nOverlap between Training and Validation sets: {len(overlap_train_val)} images")
print(f"Overlap between Training and Test sets: {len(overlap_train_test)} images")
print(f"Overlap between Validation and Test sets: {len(overlap_val_test)} images")

# Raise an assertion error if any intersection is not empty
assert len(overlap_train_val) == 0, f"Duplicate images found between Training and Validation sets: {overlap_train_val}"
assert len(overlap_train_test) == 0, f"Duplicate images found between Training and Test sets: {overlap_train_test}"
assert len(overlap_val_test) == 0, f"Duplicate images found between Validation and Test sets: {overlap_val_test}"

print("\nAll datasets are mutually exclusive, no image ID overlaps found.")

Total unique images in Training set: 24205
Total unique images in Validation set: 1904
Total unique images in Test set: 1604

Overlap between Training and Validation sets: 0 images
Overlap between Training and Test sets: 0 images
Overlap between Validation and Test sets: 0 images

All datasets are mutually exclusive, no image ID overlaps found.


Датасеты не пересекаются, отлично, сохраняем в файлы


In [19]:
import os
import pandas as pd

# 10. Convert the lists into Pandas DataFrames
train_df = pd.DataFrame(train_data, columns=['image_id', 'identity'])
val_df = pd.DataFrame(val_data, columns=['image_id', 'identity'])
test_df = pd.DataFrame(test_data, columns=['image_id', 'identity'])

# 11. Create the output directory if it doesn't exist
output_dir = os.path.join('celeba_dataset', 'fr')
os.makedirs(output_dir, exist_ok=True)

# 12. Save each DataFrame to a CSV file in the specified directory
train_output_path = os.path.join(output_dir, 'fr_train_identity.csv')
val_output_path = os.path.join(output_dir, 'fr_val_identity.csv')
test_output_path = os.path.join(output_dir, 'fr_test_identity.csv')

train_df.to_csv(train_output_path, index=False)
val_df.to_csv(val_output_path, index=False)
test_df.to_csv(test_output_path, index=False)

print(f"Training dataset saved to: {train_output_path}")
print(f"Validation dataset saved to: {val_output_path}")
print(f"Test dataset saved to: {test_output_path}")

print("First 5 rows of Training DataFrame:")
print(train_df.head())
print("\nFirst 5 rows of Validation DataFrame:")
print(val_df.head())
print("\nFirst 5 rows of Test DataFrame:")
print(test_df.head())

Training dataset saved to: celeba_dataset/fr/fr_train_identity.csv
Validation dataset saved to: celeba_dataset/fr/fr_val_identity.csv
Test dataset saved to: celeba_dataset/fr/fr_test_identity.csv
First 5 rows of Training DataFrame:
     image_id  identity
0  170472.jpg      2119
1  166806.jpg      2119
2  182036.jpg      2119
3  168392.jpg      2119
4  181507.jpg      2119

First 5 rows of Validation DataFrame:
     image_id  identity
0  095054.jpg      3094
1  033123.jpg      3094
2  131344.jpg      3563
3  111877.jpg      3563
4  031819.jpg      4019

First 5 rows of Test DataFrame:
     image_id  identity
0  172475.jpg      2490
1  166181.jpg      2490
2  111158.jpg      8845
3  019284.jpg      8845
4  011942.jpg      7967


## Сводка:

### Основные выводы анализа данных
*   Обучающий набор данных содержит 24 205 уникальных изображений, валидационный набор данных содержит 1904 уникальных изображения, а тестовый набор данных состоит из 1604 уникальных изображений.
*   Все наборы данных (обучающий, валидационный и тестовый) являются взаимоисключающими, с 0 изображениями, перекрывающимися между обучающим и валидационным наборами, 0 между обучающим и тестовым наборами и 0 между валидационным и тестовым наборами.
*   Отдельные CSV-файлы с именами `fr_train_identity.csv`, `fr_val_identity.csv` и `fr_test_identity.csv` были успешно созданы и сохранены в директории `celeba_dataset/fr/`, каждый из которых содержит столбцы `image_id` и `identity`.
